# PS 8
## Daryl Larsen
### Question 3

In [1]:
using LinearAlgebra, Random, Distributions, PrettyTables, Optim

In [2]:
function data(n; β=[1 , 1], π=[1 , 1], ρ=0.99)
    """
    n:= sample size
    β:= true beta
    π:= true pi
    ρ:= correlation coefficient
    *** I'm not even using these in the function, but I would if I wanted to costumize the function
    """
    
    # Generate V and U a according to N([0,0] ,[1 ρ ; ρ 1])
    COVAR = [1 0.99; 0.99 1]; #Var-covar matrix 
    E = randn(n,2)*sqrt(COVAR);
    U = E[:, 1]
    V=E[:, 2]
    
    X2 = randn(n,1)
    Z = randn(n,1)
        
    X1= X2 .+ Z .+ V
    Y = sin.(X1 .+ X2) .+ U

    return ( Y = Y, X1 = X1, X2 = X2, Z = Z)
end

function CriterionFV(β, Y, X1, X2, Z, n)
    ZX =  [X2 Z];
    X1hat = ZX*inv(ZX'*ZX)*ZX'*X1;
    Q = (1/(2*n))*sum((Y .- sin.(X1hat.*β[1] .+ X2.*β[2])).^2);
    return Q
end

function CriterionCF(β, Y, X1, X2, Z, n)
    ZX =  [X2 Z]
    Vhat = X1.-ZX*inv(ZX'*ZX)*ZX'*X1;
    Q = (1/(2*n))*sum((Y .- sin.(X1.*β[1] .+ X2.*β[2]) .- Vhat.*β[3]).^2);
    return Q
end

CriterionCF (generic function with 1 method)

#### (a) : Monte Carlo with n=100

In [3]:
n=100
rep=1000
biasFV=0
biasCF=0
rmseFV=0
rmseCF=0
for i in 1:rep
    (Y, X1, X2, Z) = data(n)
    resultFV = optimize(b->CriterionFV(b, Y, X1, X2, Z, n)[1,1],[0.9 ; 0.9])
    βFV = Optim.minimizer(resultFV)

    resultCF = optimize(b->CriterionCF(b, Y, X1, X2, Z, n)[1,1],[0.9 ; 0.9; 0.9])
    βCF = Optim.minimizer(resultCF)
    
    biasFV += βFV[1] -1;
    rmseFV += (βFV[1] -1)^2;
    biasCF += βCF[1] -1;
    rmseCF += (βCF[1] -1)^2;
end

biasFV = biasFV/rep
biasCF = biasCF/rep
rmseFV= sqrt(rmseFV/rep)
rmseCF= sqrt(rmseCF/rep)

table = [ "Bias" biasFV biasCF ;
"RMSE" rmseFV rmseCF ]
pretty_table(table , ["" "Fitted Value" "Control function"])

┌──────┬──────────────┬──────────────────┐
│      │ Fitted Value │ Control function │
├──────┼──────────────┼──────────────────┤
│ Bias │  -0.00389699 │       0.00146483 │
│ RMSE │     0.341415 │        0.0318837 │
└──────┴──────────────┴──────────────────┘


#### (b) : Monte Carlo with n=1000

In [4]:
n=1000
rep=1000
biasFV=0
biasCF=0
rmseFV=0
rmseCF=0
for i in 1:rep
    (Y, X1, X2, Z) = data(n)
    resultFV = optimize(b->CriterionFV(b, Y, X1, X2, Z, n)[1,1],[0.9 ; 0.9])
    βFV = Optim.minimizer(resultFV)

    resultCF = optimize(b->CriterionCF(b, Y, X1, X2, Z, n)[1,1],[0.9 ; 0.9; 0.9])
    βCF = Optim.minimizer(resultCF)
    
    biasFV += βFV[1] -1;
    rmseFV += (βFV[1] -1)^2;
    biasCF += βCF[1] -1;
    rmseCF += (βCF[1] -1)^2;
end

biasFV = biasFV/rep
biasCF = biasCF/rep
rmseFV= sqrt(rmseFV/rep)
rmseCF= sqrt(rmseCF/rep)

table = [ "Bias" biasFV biasCF ;
"RMSE" rmseFV rmseCF ]
pretty_table(table , ["" "Fitted Value" "Control function"])

┌──────┬──────────────┬──────────────────┐
│      │ Fitted Value │ Control function │
├──────┼──────────────┼──────────────────┤
│ Bias │   0.00113446 │     -0.000336139 │
│ RMSE │    0.0863977 │       0.00836604 │
└──────┴──────────────┴──────────────────┘


#### (c) : Consistency
Both estimators are consistent. The control function works better overall because the fitted value estimator takes more time to converge and the RMSE for the fitted value estimator is 10x the control function estimator RMSE.